## 1 Imports

In [33]:
import fcntl
import time
import random
import struct
import sys
import termios
import string
import os
from sympy import *
from colors import *
from IPython.display import display
import ipywidgets as widgets
import numpy as np
from numpy.linalg import inv
from pyfiglet import figlet_format

## 2 Functions

In [2]:
def printEquations(matrice_A, matrice_B, nbrEquation, nbrUnknowns, unknowns_X):
    for x in range(0, nbrEquation):
        equation = 'Equation ' + str(x + 1) + ' : '
        for y in range(0, nbrUnknowns):
            if y != nbrUnknowns - 1:
                equation = equation + str(matrice_A[x][y]) + '' +  unknowns_X[y] + ' + '
            else:
                equation = equation + str(matrice_A[x][y]) + '' + unknowns_X[y]  + ' = '
        equation = equation + str(matrice_B[x])
        print(color(equation, fg='blue'))

        
def formatEquations(matrice_A, matrice_B,nbrUnknowns, unknowns_X, equationNbr):
    equation = ''
    x = equationNbr
    for y in range(0, nbrUnknowns):
        if y == 0:
            if matrice_A[x][y] > 0: 
                equation = str(matrice_A[x][y]) + '*' + unknowns_X[y]
            elif matrice_A[x][y] < 0: 
                equation = '-' + str(matrice_A[x][y]) + '*' + unknowns_X[y]
            elif matrice_A[x][y] == 0: 
                continue
        elif y > 0 and y < nbrUnknowns :
            if matrice_A[x][y] > 0: 
                equation = equation + '+' +str(matrice_A[x][y]) + '*' + unknowns_X[y]
            elif matrice_A[x][y] < 0: 
                equation = equation + '-' +str(matrice_A[x][y]) + '*' + unknowns_X[y] 
            elif matrice_A[x][y] == 0:
                continue
    if matrice_B[x] > 0: 
        equation = equation + '-' + str(matrice_B[x])
    elif matrice_B[x] < 0: 
        equation = +equation + '+' + str(matrice_B[x])
    return equation

## 3 Declarations

In [3]:
# Declaring the variables of the problem
matrice_B = []  # Matrix to the right
const_possible_unknowns = ['x', 'y', 'z', 'w', 'v', 'u']  # Solve equation system with 6 potentialy unknowns variables
unknowns_X = []  # Keep here the effective unknowns

# Declaring the variables containing the user inputs
nbrEquation = 0
nbrUnknowns = 0
leftFormulas = []  # Array of array Containing the left side of equation
rightAnswer = []

## 4 Input : nbrEquations & nbrVariables

In [4]:
# Ask for the number of equations
print(color('Combien y-a-t-il d\'equation dans le système ?', fg='red'))
#nbrEquation = int(input())

nbrEquation = widgets.IntText(
    value=0,
    description='Nbr Equations :',
    disabled=False
)
display(nbrEquation)
print('')
# Ask for the number of Unknowns
print(color("Combien y-a-t-il de variable dans le système ?", fg='red'))
#nbrUnknowns = int(input())

nbrUnknowns = widgets.IntText(
    value=0,
    description='NbrVariable :',
    disabled=False
)
display(nbrUnknowns)
print('')

Combien y-a-t-il d'equation dans le système ?


IntText(value=0, description='Nbr Equations :')


Combien y-a-t-il de variable dans le système ?


IntText(value=0, description='NbrVariable :')

## 5 Input : Coeficient Values 

In [5]:
# Matrix initialisation (because there is not such a thing as dynamic array in python ?
matrice_A = [[0 for x in range(nbrUnknowns.value)] for y in range(nbrEquation.value)]   # Matrix to the left
matrice_A_input = [[0 for x in range(nbrUnknowns.value)] for y in range(nbrEquation.value)]
matrice_B = [0 for x in range(nbrEquation.value)] # Array (single column matrix)
matrice_B_input = [0 for x in range(nbrEquation.value)]

# Resume number of variables and equations
print(color('Il y a '+ str(nbrEquation.value) +' equations dans le système.', fg='blue'))

# We assign a variables according to the number selected 
unknowns_X =[]
string_unknowns = ''
for x in range(0, nbrUnknowns.value):
    unknowns_X.append(const_possible_unknowns[x])
    if nbrUnknowns.value == 1 : 
        string_unknowns = unknowns_X[x] + " fait partie des inconnues présentes dans le système."
        break;
    else:
        if x < nbrUnknowns.value -1 :
            string_unknowns = string_unknowns + unknowns_X[x] + ', '
        else :
            string_unknowns = string_unknowns + unknowns_X[x] + " font partie des inconnues présentes dans le système."

# Print number of variable + the ones selected
print(color("Il y a " + str(len(unknowns_X)) + " variable(s) dans le système d'équation.", fg='blue'))
print(color(string_unknowns, fg='blue'))
time.sleep(5)
print(' ')

# Enter user input for definition of equation
for x in range(0, nbrEquation.value):
    print(color('Veuillez completer l\'equation ' + str(x + 1) + ':', fg='yellow', style='underline'))
    time.sleep(5)
    for y in range(0, nbrUnknowns.value):
        print(color('Quel est le coeficient de ' + unknowns_X[y] + ' ?', fg='red'))
        
        matrice_A_input[x][y] = widgets.IntText(
            value=0,
            description= unknowns_X[y] + ' (Eq ' + str(x+1) + ') :',
            disabled=False
            )
        display(matrice_A_input[x][y])
       
    print(color('Quel est le résultat de cette équation ?', fg='red'))
    matrice_B_input[x] = widgets.IntText(
        value=0,
        description= 'Résultat (Eq ' + str(x+1) + ') :',
        disabled=False
    )
    display(matrice_B_input[x])
    print(' ')

    #matrice_B[x] = int(input())

Il y a 2 equations dans le système.
Il y a 2 variable(s) dans le système d'équation.
x, y font partie des inconnues présentes dans le système.
 
Veuillez completer l'equation 1:
Quel est le coeficient de x ?


IntText(value=0, description='x (Eq 1) :')

Quel est le coeficient de y ?


IntText(value=0, description='y (Eq 1) :')

Quel est le résultat de cette équation ?


...

IntText(value=0, description='x (Eq 2) :')

Quel est le coeficient de y ?


IntText(value=0, description='y (Eq 2) :')

Quel est le résultat de cette équation ?


IntText(value=0, description='Résultat (Eq 2) :')

## 6 Output : Equation et égalité

In [35]:
for x in range(0, nbrEquation.value):
    for y in range(0, nbrUnknowns.value):
        matrice_A[x][y] = matrice_A_input[x][y].value 
    matrice_B[x] = matrice_B_input[x].value

#matrice_X =  [0 for x in range(nbrUnknowns.value)]
#for x in range(nbrEquation.value):
#    matrice_X.append
    
printEquations(matrice_A, matrice_B, nbrEquation.value, nbrUnknowns.value, unknowns_X)
print()
print(color(" ==>", fg='red'))
print()
matrice_A = np.array(matrice_A, dtype='int')
matrice_B = np.array(matrice_B, dtype='int')
unknowns_X = np.array(unknowns_X, dtype='str')
print(color("A", fg='blue')+ color(" = ", fg='red') + color(str(matrice_A), fg='blue'))
print(color("B", fg='blue')+ color(" = ", fg='red') + color(str(matrice_B), fg='blue'))
print(color("X", fg='blue')+ color(" = ", fg='red') + color(str(unknowns_X), fg='blue'))
print()
print(color(" ==>", fg='red'))
print()
print(color("AX=B", fg='blue'))
print()
print(color(" <==>", fg='red'))
print()
print(color(str(matrice_A), fg='blue') + color(".", fg='red') + color(str(unknowns_X), fg='blue')+ color(" = ", fg='red')+ color(str(matrice_B), fg='blue') )

Equation 1 : 1x + 1y = 58
Equation 2 : 4x + 2y = 160

 ==>

A = [[1 1]
 [4 2]]
B = [ 58 160]
X = ['x' 'y']

 ==>

AX=B

 <==>

[[1 1]
 [4 2]].['x' 'y']=[ 58 160]


## 7 Inverse, Simplification et solution version 1

In [45]:
print(color("Simplification du coeficient de X...(A)", fg="blue"))
print(color("Multiplication de AX par l'inverse de A", fg="blue"))


# Matrice inverse
matrice_A_inv = inv(matrice_A)

print(color("inv(A).A.X", fg='blue') + color("=", fg='red') + color("inv(A).B", fg='blue') )
print()
print(color("Inverse de A", fg='blue'))
print(color("inv(A)", fg='blue') + color(" = ", fg='red') + color(str(matrice_A_inv), fg='blue'))

# Multiplication inverse(A).B
inv_A_muliplie_mat_b = matrice_A_inv.dot(matrice_B)

print(color("X ", fg='blue') + color(" = ", fg="red") + color(str(inv_A_muliplie_mat_b), fg="blue"))
print()
print(color("Solution", fg='blue'))
for i in range(0,nbrUnknowns.value):
    print(color(str(unknowns_X[i]) + " = " + str(inv_A_muliplie_mat_b[i]), fg='blue'))

Simplification du coeficient de X...(A)
Multiplication de AX par l'inverse de A
inv(A).A.X=inv(A).B

Inverse de A
inv(A) = [[-1.   0.5]
 [ 2.  -0.5]]
X  = [22. 36.]

Solution
x = 22.0
y = 36.0


## 8 Output : Solution version 2

In [123]:
# Cast to numpy arrays of int
soluce = np.linalg.solve(matrice_A, matrice_B)   # <== C'EST ICI QUE CA SE PASSE ! Mouahaha...
for i in range(0,nbrUnknowns.value):
    print(color(str(unknowns_X[i]) + " = " + str(soluce[i]), fg='blue'))


x = 22.0
y = 36.0
